# PySpark 02 — Spark Transformations
This notebook focuses on common transformations in PySpark:

1. Selecting columns
2. Filtering rows
3. Adding new columns
4. Renaming or dropping columns
5. Sorting data
6. Grouping & Aggregation
7. Chaining multiple transformations

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

# Create SparkSession
spark = SparkSession.builder \
    .appName("Spark Transformations") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 17:26:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/05 17:26:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [45]:
# Sample data
raw_data = [
    ("Alice", 25, "New York"),
    ("Bob", 32, "Los Angeles"),
    ("Cathy", 19, "Chicago"),
    ("Dan", 40, "New York")
]
columns = ["name", "age", "city"]

df = spark.createDataFrame(raw_data, columns)

# 1. Selecting Columns
You can use `.select()` to return a new DataFrame with only the specified columns.

## 1.1 Select all (*)

In [4]:
df.select("*").show()

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|Cathy| 19|    Chicago|
|  Dan| 40|   New York|
+-----+---+-----------+



## 1.2 Select by Column Name (String)

In [5]:
selected = df.select("name", "age")
selected.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|  Bob| 32|
|Cathy| 19|
|  Dan| 40|
+-----+---+



## 1.3 Select Using `col()` (Explicit)

In [6]:
df.select(col("name"), col("age")).show()

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|  Bob| 32|
|Cathy| 19|
|  Dan| 40|
+-----+---+



## 1.4 Select with Aliased Columns (Renaming)

In [7]:
df.select(
    col("name").alias("full_name"),
    col("age").alias("years_old")
).show()

+---------+---------+
|full_name|years_old|
+---------+---------+
|    Alice|       25|
|      Bob|       32|
|    Cathy|       19|
|      Dan|       40|
+---------+---------+



# 2. Filtering Rows
Use `.filter()` or `.where()` to return a new DataFrame with rows that meet a condition.

| Method      | Style                      |
| ----------- | -------------------------- |
| `.filter()` | More common in Python code |
| `.where()`  | Familiar to SQL users      |


## 2.1 Basic Filter with Column Condition

In [8]:
df.filter(col("age") > 30).show()

+----+---+-----------+
|name|age|       city|
+----+---+-----------+
| Bob| 32|Los Angeles|
| Dan| 40|   New York|
+----+---+-----------+



In [9]:
df.where("age > 30").show()

+----+---+-----------+
|name|age|       city|
+----+---+-----------+
| Bob| 32|Los Angeles|
| Dan| 40|   New York|
+----+---+-----------+



## 2.2 Filter with Multiple Conditions (AND/OR)

In [10]:
df.filter((col("age") > 20) & (col("city") == "New York")).show()
# or 
df.where('age > 20 and city = "New York"').show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+



In [11]:
df.filter((col("city") == "Chicago") | (col("city") == "New York")).show()
# or
df.where('city = "Chicago" or city = "New York"').show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|Cathy| 19| Chicago|
|  Dan| 40|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|Cathy| 19| Chicago|
|  Dan| 40|New York|
+-----+---+--------+



## 2.3 Filter with isin()

In [12]:
df.filter(col("city").isin("Chicago", "New York")).show()
# or
df.where('city in ("Chicago", "New York")').show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|Cathy| 19| Chicago|
|  Dan| 40|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|Cathy| 19| Chicago|
|  Dan| 40|New York|
+-----+---+--------+



In [14]:
cities = ["Chicago", "New York"]
df.filter(col("city").isin(cities)).show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|Cathy| 19| Chicago|
|  Dan| 40|New York|
+-----+---+--------+



## 2.4 Filter with ~ for NOT

In [13]:
df.filter(~col("city").isin("Chicago", "New York")).show()
# or 
df.where('city not in ("Chicago", "New York")')

+----+---+-----------+
|name|age|       city|
+----+---+-----------+
| Bob| 32|Los Angeles|
+----+---+-----------+



DataFrame[name: string, age: bigint, city: string]

## 2.5 Filter with `isNull()` / `isNotNull()`

In [15]:
df.filter(col("city").isNotNull()).show()
# or
df.where("city IS NOT NULL").show()

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|Cathy| 19|    Chicago|
|  Dan| 40|   New York|
+-----+---+-----------+

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|Cathy| 19|    Chicago|
|  Dan| 40|   New York|
+-----+---+-----------+



## 2.6 Filter with like / rlike (pattern matching)

In [16]:
df.filter(col("city").like("New%")).show()
# or
df.where("city LIKE 'New%'").show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+



For regex-like string matching:

In [17]:
df.filter(col("name").rlike("^[A-C].*")).show()
# or
df.where("name RLIKE '^[A-C].*'").show()

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|Cathy| 19|    Chicago|
+-----+---+-----------+

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|Cathy| 19|    Chicago|
+-----+---+-----------+



## 2.7 Filter with `.between()`

In [18]:
df.filter(col("age").between(20, 30)).show()
# or
df.where("age BETWEEN 20 AND 30").show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
+-----+---+--------+



## 2.8 Filtering on `startswith()`, `endswith()`, `contains()`

In [21]:
df.filter(col("city").startswith("New")).show()
# or
df.where('city like "New%"').show()


+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+



In [22]:
df.filter(col("name").endswith("y")).show()
# or 
df.where("name LIKE '%y'").show()

+-----+---+-------+
| name|age|   city|
+-----+---+-------+
|Cathy| 19|Chicago|
+-----+---+-------+

+-----+---+-------+
| name|age|   city|
+-----+---+-------+
|Cathy| 19|Chicago|
+-----+---+-------+



In [23]:
df.filter(col("city").contains("York")).show()
# or 
df.where("city LIKE '%York%'").show()

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+

+-----+---+--------+
| name|age|    city|
+-----+---+--------+
|Alice| 25|New York|
|  Dan| 40|New York|
+-----+---+--------+



# 3. Adding New Columns
Use `.withColumn()` to add or modify a column in a DataFrame. You can apply expressions, functions, or constants.

## 3.1 Add Column with Literal Value

In [26]:
from pyspark.sql.functions import lit

df.withColumn("country", lit("USA")).show()

+-----+---+-----------+-------+
| name|age|       city|country|
+-----+---+-----------+-------+
|Alice| 25|   New York|    USA|
|  Bob| 32|Los Angeles|    USA|
|Cathy| 19|    Chicago|    USA|
|  Dan| 40|   New York|    USA|
+-----+---+-----------+-------+



## 3.2 Add Column Based on Expression

In [28]:
from pyspark.sql.functions import expr

df.withColumn("age_in_5_years", expr("age + 5")).show()

+-----+---+-----------+--------------+
| name|age|       city|age_in_5_years|
+-----+---+-----------+--------------+
|Alice| 25|   New York|            30|
|  Bob| 32|Los Angeles|            37|
|Cathy| 19|    Chicago|            24|
|  Dan| 40|   New York|            45|
+-----+---+-----------+--------------+



## 3.3 Add Column with Multiple Conditional Logic (like if-else)

In [34]:
from pyspark.sql.functions import when, col

df.withColumn(
    "Types",
    when(col("age") >= 21, "Adult")
    .when(col("age") >= 12, "Teen")
    .otherwise("Kid")
).show()

+-----+---+-----------+-----+
| name|age|       city|Types|
+-----+---+-----------+-----+
|Alice| 25|   New York|Adult|
|  Bob| 32|Los Angeles|Adult|
|Cathy| 19|    Chicago| Teen|
|  Dan| 40|   New York|Adult|
+-----+---+-----------+-----+



## 3.4 Add Column Using Existing Columns

In [38]:
from pyspark.sql.functions import concat, col, lit

df.withColumn(
    "name_age",
    concat(col("name"), lit("_"), col("age").cast("string"))
).show()

+-----+---+-----------+--------+
| name|age|       city|name_age|
+-----+---+-----------+--------+
|Alice| 25|   New York|Alice_25|
|  Bob| 32|Los Angeles|  Bob_32|
|Cathy| 19|    Chicago|Cathy_19|
|  Dan| 40|   New York|  Dan_40|
+-----+---+-----------+--------+



## 3.5 Overwrite an Existing Column

In [32]:
df.withColumn("age", col("age") + 1).show()  # updates the 'age' column

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Alice| 26|   New York|
|  Bob| 33|Los Angeles|
|Cathy| 20|    Chicago|
|  Dan| 41|   New York|
+-----+---+-----------+



# 4. Renaming or dropping columns
You can rename or drop columns in a DataFrame using `.withColumnRenamed()` and `.drop()`.

## 4.1 Renaming Column(s)

In [41]:
# Single Column
df.withColumnRenamed("name", "full_name").show()

+---------+---+-----------+
|full_name|age|       city|
+---------+---+-----------+
|    Alice| 25|   New York|
|      Bob| 32|Los Angeles|
|    Cathy| 19|    Chicago|
|      Dan| 40|   New York|
+---------+---+-----------+



In [42]:
# Multiple Column
df.withColumnRenamed("name", "full_name") \
  .withColumnRenamed("age", "years_old") \
  .show()

+---------+---------+-----------+
|full_name|years_old|       city|
+---------+---------+-----------+
|    Alice|       25|   New York|
|      Bob|       32|Los Angeles|
|    Cathy|       19|    Chicago|
|      Dan|       40|   New York|
+---------+---------+-----------+



## 4.2 Dropping Column(s)

In [43]:
# Single Column
df.drop("city").show()

+-----+---+
| name|age|
+-----+---+
|Alice| 25|
|  Bob| 32|
|Cathy| 19|
|  Dan| 40|
+-----+---+



In [44]:
df.drop("city", "age").show()

+-----+
| name|
+-----+
|Alice|
|  Bob|
|Cathy|
|  Dan|
+-----+



# 5. Sorting Data

Use `.orderBy()` or `.sort()` to sort DataFrames by one or more columns.

Both methods behave the same — `.orderBy()` is preferred for clarity.

## 5.1 Sort by One Column (Ascending)

In [47]:
df.sort("age").show()
# or
df.orderBy(col("age").asc()).show()

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Cathy| 19|    Chicago|
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|  Dan| 40|   New York|
+-----+---+-----------+

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Cathy| 19|    Chicago|
|Alice| 25|   New York|
|  Bob| 32|Los Angeles|
|  Dan| 40|   New York|
+-----+---+-----------+



## 5.2 Sort by One Column (Descending)

In [48]:
df.orderBy(col("age").desc()).show()


+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|  Dan| 40|   New York|
|  Bob| 32|Los Angeles|
|Alice| 25|   New York|
|Cathy| 19|    Chicago|
+-----+---+-----------+



## 5.3 Sort by Multiple Columns

In [49]:
df.orderBy(col("city").asc(), col("age").desc()).show()

+-----+---+-----------+
| name|age|       city|
+-----+---+-----------+
|Cathy| 19|    Chicago|
|  Bob| 32|Los Angeles|
|  Dan| 40|   New York|
|Alice| 25|   New York|
+-----+---+-----------+



# 6. Grouping & Aggregation

Use `.groupBy()` with aggregation functions to summarize your data.

Import functions from `pyspark.sql.functions`.

## 6.1 Group By One Column and Count

In [50]:
df.groupBy("city").count().show()

+-----------+-----+
|       city|count|
+-----------+-----+
|Los Angeles|    1|
|   New York|    2|
|    Chicago|    1|
+-----------+-----+



## 6.2 Group By and Aggregate Multiple Columns

In [53]:
from pyspark.sql.functions import avg, max, min

df.groupBy("city").agg(
    avg("age").alias("avg_age"),
    max("age").alias("max_age"),
    min("age").alias("min_age")
).show()

+-----------+-------+-------+-------+
|       city|avg_age|max_age|min_age|
+-----------+-------+-------+-------+
|Los Angeles|   32.0|     32|     32|
|   New York|   32.5|     40|     25|
|    Chicago|   19.0|     19|     19|
+-----------+-------+-------+-------+



## 6.3 Group By Multiple Columns


In [55]:
df.groupBy("city", "age").count().show()

+-----------+---+-----+
|       city|age|count|
+-----------+---+-----+
|Los Angeles| 32|    1|
|   New York| 25|    1|
|   New York| 40|    1|
|    Chicago| 19|    1|
+-----------+---+-----+



## 6.4 Filter After Grouping (using .filter())

In [54]:
df.groupBy("city").count().filter(col("count") > 1).show()

+--------+-----+
|    city|count|
+--------+-----+
|New York|    2|
+--------+-----+



## 6.5 Order Aggregated Results

In [57]:
df.groupBy("city").avg("age").orderBy("avg(age)", ascending=False).show()

+-----------+--------+
|       city|avg(age)|
+-----------+--------+
|   New York|    32.5|
|Los Angeles|    32.0|
|    Chicago|    19.0|
+-----------+--------+



# 7. Chaining Multiple Transformations
In PySpark, you can chain multiple operations together in a fluent, readable way.

This avoids creating intermediate DataFrames and keeps your logic clear.

## 7.1 Example: Filter → Add Column → Sort → Show


In [58]:
df.filter(col("age") > 20) \
  .withColumn("country", lit("USA")) \
  .orderBy(col("age").desc()) \
  .show()

+-----+---+-----------+-------+
| name|age|       city|country|
+-----+---+-----------+-------+
|  Dan| 40|   New York|    USA|
|  Bob| 32|Los Angeles|    USA|
|Alice| 25|   New York|    USA|
+-----+---+-----------+-------+



## 7.2 Another Example: Filter Teens → Add Flag → Drop Column

In [59]:
df.filter((col("age") >= 13) & (col("age") <= 19)) \
  .withColumn("group", lit("Teen")) \
  .drop("city") \
  .show()

+-----+---+-----+
| name|age|group|
+-----+---+-----+
|Cathy| 19| Teen|
+-----+---+-----+



In [60]:
spark.stop()